<a href="https://colab.research.google.com/github/elliekim9881/AIFFELthon_I15/blob/main/keywords/keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install konlpy

In [43]:
import networkx as nx
from itertools import combinations
from konlpy.tag import Okt
import re
import pandas as pd

In [44]:
!pip install xlrd

In [45]:
file_path = '/content/1_30000_20231101.xls'
df = pd.read_excel(file_path)
df.head()

,표제어,동형어 번호,구분,품사,고유어 여부,원어,발음,활용,파생어,☞(가 보라),...,러시아어 부표제어 대역어,러시아어 부표제어 대역어 뜻풀이,영어 부표제어 대역어,영어 부표제어 대역어 뜻풀이,일본어 부표제어 대역어,일본어 부표제어 대역어 뜻풀이,프랑스어 부표제어 대역어,프랑스어 부표제어 대역어 뜻풀이,스페인어 부표제어 대역어,스페인어 부표제어 대역어 뜻풀이
0,가,1,단어,명사,고유어,NaN,가ː,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,가,1,단어,명사,고유어,NaN,가ː,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-가,1,단어,접사,한자어,哥,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,가-,1,단어,접사,한자어,假,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,가,2,단어,명사,한자어,可,가ː,NaN,가하다2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
# '-'을 포함하거나 글자 수가 1인 행 삭제
df = df[~(df['표제어'].str.contains('-') | (df['표제어'].str.replace('-', '').str.len() <= 1))]

In [47]:
df = df[['표제어', '의미 범주', '뜻풀이', '용례']]
df.head()

,표제어,의미 범주,뜻풀이,용례
20,가가호호,NaN,한 집 한 집.,<구> 가가호호를 돌다.\n<구> 가가호호를 찾아다니다.\n<구> 가가호호를 방문하...
21,가가호호,NaN,집집마다.,<구> 가가호호 돌아다니다.\n<구> 가가호호 들르다.\n<구> 가가호호 방문하다....
22,가감,NaN,더하거나 뺌.,<구> 내용의 가감.\n<구> 가감이 없다.\n<구> 가감을 하다.\n<문장> 말을...
23,가감승제,NaN,"더하기, 빼기, 곱하기, 나누기의 계산.",<구> 가감승제 계산.\n<구> 가감승제 기호.\n<구> 가감승제 문제.\n<구> ...
24,가감하다,NaN,더하거나 빼다.,<구> 가감한 금액.\n<구> 가감한 돈.\n<구> 말을 가감하다.\n<구> 양념을...


불용어 제거

In [48]:
pip install nltk

In [49]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [50]:
from nltk.tokenize import word_tokenize

# 1. txt 파일에서 불용어 목록을 읽어오기
with open('/content/korean_stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = file.read().splitlines()

# 2. 읽어온 불용어 목록을 리스트로 변환

# '뜻풀이', '용례' 열의 각 문장에서 불용어를 제거하는 함수
def remove_stopwords(text):
    if isinstance(text, str):  # text가 문자열인 경우에만 처리
        word_tokens = word_tokenize(text)
        result = [w for w in word_tokens if w not in stop_words]
        return " ".join(result)
    else:
        return text  # NaN 또는 비어 있는 문자열인 경우 그대로 반환

# '뜻풀이' , '용례' 열에 불용어 처리 적용
df['뜻풀이'] = df['뜻풀이'].apply(remove_stopwords)
df['용례'] = df['용례'].apply(remove_stopwords)

# 결과 출력
df.head()

,표제어,의미 범주,뜻풀이,용례
20,가가호호,NaN,한 집 한 집,가가호호를 돌다 가가호호를 찾아다니다 가가호호를 방문하다 가가호호에 공급하다 가가호...
21,가가호호,NaN,집집마다,가가호호 돌아다니다 가가호호 들르다 가가호호 방문하다 가가호호 인사하다 가가호호 찾...
22,가감,NaN,더하거나 뺌,내용의 가감 가감이 없다 가감을 하다 문장 말을 잘하는 사람은 상황에 맞게 말을 가...
23,가감승제,NaN,더하기 빼기 곱하기 나누기의 계산,가감승제 계산 가감승제 기호 가감승제 문제 가감승제 연산 가감승제를 구하다 가감승제...
24,가감하다,NaN,더하거나 빼다,가감한 금액 가감한 돈 말을 가감하다 양념을 가감하다 월급을 가감하다 문장 요리사는...


In [51]:
from konlpy.tag import Okt
okt = Okt()

In [52]:
# '뜻풀이' 열의 주요 단어 및 핵심 단어 추출 및 '정답 후보' 열 추가
def extract_keywords(text):
    if isinstance(text, str):
        # 형태소 분석
        morphemes = okt.pos(text)
        # 명사 및 핵심 단어 추출
        keywords = [word for word, pos in morphemes if pos in ['Noun', 'Adjective', 'Verb', 'Adverb']]
        # 중복을 제거하여 리스트로 변환
        unique_keywords = list(set(keywords))
        return ' '.join(unique_keywords)
    else:
        return text  # NaN 또는 비어 있는 문자열인 경우 그대로 반환

In [53]:
# '뜻풀이' 열에 주요 및 핵심 단어 추출 함수 적용
df['정답 후보'] = df['뜻풀이'].apply(extract_keywords)

# 결과 출력
df.head()

,표제어,의미 범주,뜻풀이,용례,정답 후보
20,가가호호,NaN,한 집 한 집,가가호호를 돌다 가가호호를 찾아다니다 가가호호를 방문하다 가가호호에 공급하다 가가호...,집 한
21,가가호호,NaN,집집마다,가가호호 돌아다니다 가가호호 들르다 가가호호 방문하다 가가호호 인사하다 가가호호 찾...,집집
22,가감,NaN,더하거나 뺌,내용의 가감 가감이 없다 가감을 하다 문장 말을 잘하는 사람은 상황에 맞게 말을 가...,더하거나 뺌
23,가감승제,NaN,더하기 빼기 곱하기 나누기의 계산,가감승제 계산 가감승제 기호 가감승제 문제 가감승제 연산 가감승제를 구하다 가감승제...,곱하기 계산 빼기 더하기 나누기
24,가감하다,NaN,더하거나 빼다,가감한 금액 가감한 돈 말을 가감하다 양념을 가감하다 월급을 가감하다 문장 요리사는...,더하거나 빼다


In [54]:
# '정답 후보' 열에서 가장 대표적인 명사 선택 함수
def select_representative_noun(text):
    if isinstance(text, str) and text:  # 비어 있지 않은 문자열만 처리
        # 형태소 분석을 통해 명사 추출
        morphemes = okt.pos(text)
        nouns = [word for word, pos in morphemes if pos == 'Noun']
        if nouns:  # 명사가 있는 경우에만 가장 높은 빈도수를 가진 명사 선택
            word_counts = {}
            for word in nouns:
                if word in word_counts:
                    word_counts[word] += 1
                else:
                    word_counts[word] = 1
            main_noun = max(word_counts, key=word_counts.get)
            return main_noun
    return text  # NaN 또는 빈 문자열인 경우 그대로 반환

In [55]:
# '정답' 열에 가장 대표적인 명사 선택 함수 적용
df['정답'] = df['정답 후보'].apply(select_representative_noun)

# 결과 출력
df.sample(n=30, replace=True)

,표제어,의미 범주,뜻풀이,용례,정답 후보,정답
2395,개활하다,NaN,장소가 앞이 막힘없이 탁 트여 시원하고 넓다,개활한 대지 땅이 개활하다 바다가 개활하다 사막이 개활하다 조망이 개활하다 평야가 ...,트여 앞 장소 시원하고 막힘 넓다 탁 없이,앞
28401,발흥,NaN,국가 이념 세력 등이 갑자기 일어나 힘을 얻음,국가의 발흥 문화의 발흥 민족주의의 발흥 예술의 발흥 종교의 발흥 학문의 발흥 발흥...,이념 힘 국가 갑자기 등 일어나 얻음 세력,이념
14530,누그러지다,NaN,화가 있거나 경직되고 흥분해 있던 마음 등이 부드러워지거나 약해지다,마음이 누그러지다 분위기가 누그러지다 성미가 누그러지다 적대감이 누그러지다 표정이 ...,마음 흥분해 경직 되고 등 부드러워지거나 화가 있거나 있던 약해지다,마음
15794,다음,개념 > 순서,차례의 뒤,말 번 사람 순서 역 정거장 차례 문장 사회자는 노래할 사람을 소개했다 문장 학생들...,뒤 차례,뒤
5708,공로자,NaN,일을 성사시키는 데 공을 세운 사람,제일 공로자 최고 공로자 최대 공로자 숨은 공로자 공로자 표창 공로자를 선정하다 공...,세운 사람 공 시키는 일 성사 데,세운
22525,마찰하다,NaN,사람들이나 집단의 생각 또는 의견이 달라서 충돌하다,문장 왕과 신하가 크게 마찰하여 조정에 소란이 일어났다 문장 우리나라는 미국과 무역...,의견 사람 집단 달라 또는 충돌 생각 하다,의견
11869,나다,NaN,햇빛 등이 나타나다,해가 나다 햇빛이 나다 문장 낮에 햇빛이 나서 안개가 사라졌다 문장 며칠 해가 나지...,등 햇빛 나타나다,등
8256,귀중품,NaN,값비싸고 귀한 물건,고가의 귀중품 귀중품 도난 귀중품을 다루다 귀중품을 맡기다 귀중품을 보관하다 귀중품...,물건 귀한 비싸고 값,물건
7872,굴리다,NaN,이득을 보기 돈을 투자하거나 거래하다,돈을 굴리다 예산을 굴리다 유산을 굴리다 자금을 굴리다 큰돈을 굴리다 문장 그는 돈...,거나 투자 돈 보기 거래 이득 하다,거나
19092,동토,NaN,얼어붙은 땅,동토 지대 동토 지반 동토가 녹다 동토가 되다 동토가 형성되다 문장 이렇게 땅이 차...,땅 얼어붙은,땅
